# Integrácia dát

In [7]:
from pyspark.sql import SparkSession

#### Vytvorenie SparkSession

In [8]:
spark = SparkSession.builder.appName("car_accidents").getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("ERROR")

#### Načítanie dát

In [ ]:
df_accidents = spark.read.csv("../data.tmp/CarAccidents/Accidents.csv", header=True, inferSchema=True)
df_casualties = spark.read.csv("../data.tmp/CarAccidents/Casualties.csv", header=True, inferSchema=True)
df_vehicles = spark.read.csv("../data.tmp/CarAccidents/Vehicles.csv", header=True, inferSchema=True)
df_vehicles = df_vehicles.drop("Vehicle_Reference")

df_accidents.show(5)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/c:/Users/jakub/Desktop/4LS/TSVD/BDP_Zadanie/Big-Data-Processing/data/CarAccidents/Accidents.csv.

#### Spojenie dát podľa Accident_Index

In [ ]:
df = df_accidents.join(df_casualties, ["Accident_Index"], "full")
df = df.join(df_vehicles, ["Accident_Index"], "full")
df.show(5)

+--------------+---------------------+----------------------+---------+---------+------------+-----------------+------------------+--------------------+----------+-----------+-------------------+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+-----------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+-------------+----------------------+----------------------------------+-------------+-----------------------+------------+-----------------------+-----------------+--------------------------------+---

#### Vytvorenie vzorky 10% dát stratifikovane

In [ ]:
df_sampled = df.sampleBy("Accident_Severity", fractions={1: 0.1, 2: 0.1, 3: 0.1}, seed=1234)

In [ ]:
df.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()
df_sampled.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

+-----------------+-------+
|Accident_Severity|  count|
+-----------------+-------+
|                1|  83607|
|                2| 596571|
|                3|3607415|
+-----------------+-------+



+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  8145|
|                2| 59787|
|                3|360841|
+-----------------+------+



#### Rozdelenie dát na trénovaciu a testovaciu množinu

In [ ]:
df_train, df_test = df_sampled.randomSplit([0.70, 0.30], seed=1234)

In [ ]:
df_train.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()
df_test.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

[6360.969s][warning][gc,alloc] Executor task launch worker for task 3.0 in stage 777.0 (TID 2246): Retried waiting for GCLocker too often allocating 1048578 words


+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  5748|
|                2| 41617|
|                3|252928|
+-----------------+------+



+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  2508|
|                2| 17952|
|                3|107855|
+-----------------+------+

